# Projekt Terawatt

*von Team Coding Agents*


----


## Simulation

In [1]:
import iplantuml
import datetime
import terawatt_model

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
# run at the start of every ipython notebook to use plotly.offline
# this injects the plotly.js source files into the notebook
init_notebook_mode(connected=True)

from IPython.display import IFrame
import numpy

### System with set of input and outputs

In [26]:
import random

# Timeconsuming calculation

sun = terawatt_model.Sun()
photovoltaic = terawatt_model.Photovoltaic()
car1 = terawatt_model.Car()
car2 = terawatt_model.Car()
provider = terawatt_model.Provider()
battery = terawatt_model.Battery()
electrolysis = terawatt_model.Electrolysis()
methanization = terawatt_model.Methanization()
cogeneration = terawatt_model.Cogeneration()
time_start = datetime.datetime(2016, 9, 24, 0, 0, 0, 0)

start = dict(
    car1_energy = random.randint(0, car1.energy_max.electrical),
    charge_car1 = random.randint(0, 19),
    car2_energy = random.randint(0, car2.energy_max.electrical),
    charge_car2 = random.randint(0, 20),
    battery_energy = random.randint(0, battery.energy_max.electrical),
)

# Unrandomize
start['car1_energy'] = 7000
start['charge_car1'] = 8
start['car2_energy'] = 12000
start['charge_car2'] = 18
start['battery_energy'] = 22000


car1.energy_now.electrical = start['car1_energy']
car2.energy_now.electrical = start['car2_energy']
battery.battery_current_energy = start['battery_energy']

In [27]:
print('Energy - Start of day')
print('Car 1 - electrical energy', round(car1.energy_now.electrical), 'Wh')
print('Car 1 - start of charge', start['charge_car1'])
print('Car 2 - electrical energy', round(car2.energy_now.electrical), 'Wh')
print('Car 2 - start of charge', start['charge_car2'])
print('Battery - electrical energy', round(battery.battery_current_energy), 'Wh')
print('Cogeneration - Methane', round(cogeneration.energy_now.chemical), 'Wh')
print('Provider - electrical energy', round(provider.energy_consumed.electrical), 'Wh')

Energy - Start of day
Car 1 - electrical energy 7000 Wh
Car 1 - start of charge 8
Car 2 - electrical energy 12000 Wh
Car 2 - start of charge 18
Battery - electrical energy 22000 Wh
Cogeneration - Methane 0 Wh
Provider - electrical energy 0 Wh


In [28]:
t = []
r1 = []
r2 = []
r3 = []
r4 = []
r5 = []
r6 = []
r7 = []
# Run for timestep
for dt in range(0, 3600*24-1, terawatt_model.timestep):
    time = time_start + datetime.timedelta(0,dt) # days, seconds
    
    power = sun.update(time=time)
    power = photovoltaic.update(power, time=time)
    
    power = battery.update(power, state='consume')
    power = electrolysis.update(power, state='both', power_requested=methanization.get_power_in_max())
    power = methanization.update(power, state='both', power_requested=methanization.get_power_out_max())
    power = cogeneration.update(power, state='consume')
    
    # Consumers
    power_requested = terawatt_model.Power()
    if start['charge_car1'] <= time.hour <= start['charge_car1'] + 4:
        # cannot compare against full charge. Due to incremental energies we never reach full charge exactly.
        if car1.energy_now.electrical < 0.99*car1.energy_max.electrical:
            power_requested.electrical += car1.get_power_in_max().electrical
    
    if start['charge_car2'] <= time.hour <= start['charge_car2'] + 4:
        if car2.energy_now.electrical < 0.99*car2.energy_max.electrical:
            power_requested.electrical += car2.get_power_in_max().electrical
        
    power_requested.chemical += cogeneration.power_conversion_electrical_to_chemical(power_requested.electrical).chemical
    power = cogeneration.update(power, state='provide', power_requested=power_requested)
    power_requested.electrical -= power.electrical
    power = battery.update(power, state='provide', power_requested=power_requested)

    if start['charge_car1'] <= time.hour <= start['charge_car1'] + 4:
        power = car1.update(power)
    if start['charge_car2'] <= time.hour <= start['charge_car2'] + 4:
        power = car2.update(power)

    power = provider.update(power)
    
    # Store only each minute
    if dt%60:
        t.append(time)
        r1.append(photovoltaic.power.electrical)
        r2.append(battery.energy_now.electrical)
        r3.append(car1.energy_now.electrical)
        r4.append(car2.energy_now.electrical)
        r5.append(electrolysis.energy_now.chemical)
        r6.append(cogeneration.energy_now.chemical)
        r7.append(cogeneration.power.electrical)

### System with set of input and outputs

In [29]:
print('Energy - End of day')
print('Car 1 - electrical energy', round(car1.energy_now.electrical), 'Wh')
print('Car 1 - start of charge', start['charge_car1'])
print('Car 2 - electrical energy', round(car2.energy_now.electrical), 'Wh')
print('Car 2 - start of charge', start['charge_car2'])
print('Battery - electrical energy', round(battery.energy_now.electrical), 'Wh')
print('Cogeneration - Methane', round(cogeneration.energy_now.chemical), 'Wh')
print('Provider - electrical energy', round(provider.energy_consumed.electrical), 'Wh')

Energy - End of day
Car 1 - electrical energy 21780 Wh
Car 1 - start of charge 8
Car 2 - electrical energy 21781 Wh
Car 2 - start of charge 18
Battery - electrical energy 17102 Wh
Cogeneration - Methane 2177 Wh
Provider - electrical energy 0 Wh


In [30]:
# Create a trace
trace1 = go.Scatter(
    x = t,
    y = r1,
    name = 'Photovoltaic charging power',
)
trace2 = go.Scatter(
    x = t,
    y = r2,
    name = 'Buffer battery energy',
    yaxis='y2',
)
trace3 = go.Scatter(
    x = t,
    y = r3,
    name = 'Car 1 battery energy',
    yaxis='y2',
)
trace4 = go.Scatter(
    x = t,
    y = r4,
    name = 'Car 2 battery energy',
    line = dict( color = 'black', dash = 4 ),
    yaxis='y2',
)
trace5 = go.Scatter(
    x = t,
    y = r5,
    name = 'Electrolysis chemical energy',
    line = dict( color = 'red' ),
    yaxis='y2',
)
trace6 = go.Scatter(
    x = t,
    y = r6,
    name = 'Cogeneration chemical energy',
    line = dict( color = 'violett'),
    yaxis='y2',
)
trace7 = go.Scatter(
    x = t,
    y = r7,
    name = 'Cogeneration power',
)
data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]

layout = go.Layout(
                xaxis=go.XAxis(title='Time of day'),
                yaxis=go.YAxis(title='W'),
                yaxis2=go.YAxis(
                            title='Wh',
                            #titlefont={color:'rgb(148, 103, 189)'},
                            #tickfont={color:'rgb(148, 103, 189)'},
                            overlaying='y',
                            side='right'
                )
)
fig = go.Figure(data=data, layout=layout)
plot(fig, filename='pictures/cars_delayed_charging_random_load_3.html', include_plotlyjs=True, show_link=False)

'file:///home/vagrant/what-the-data/pictures/cars_delayed_charging_random_load_3.html'

In [31]:
IFrame('pictures/cars_delayed_charging_random_load_3.html', width=900, height=500)

### Undisciplined agents    (red : not enough fuel)

In [3]:
import json
import os

with open(os.path.join('terawatt_simulation', 'agents_undisciplined.dat')) as json_data:
    data = json.load(json_data)


z = []
x = []
for i,data_agent in enumerate(data):
    c1 = []
    c2 = []
    x.append(str(data_agent[1]['car1_distance_work'])+' km '+str(data_agent[1]['charge_car1_start'])+' h') #+str(i*2))
    x.append(str(data_agent[1]['car2_distance_work'])+' km '+str(data_agent[1]['charge_car2_start'])+' h') #+str(i*2+1)) 
    for d in data_agent:
        c1.append(int(d['car1_drive_success']))
        c2.append(int(d['car2_drive_success']))
    z.append(c1)
    z.append(c2)

colorscale = [[0, 'red'], [1, '#001f3f']]
hm = [
    go.Heatmap(
        z=numpy.transpose(z),
        x=numpy.transpose(x),
        colorscale=colorscale,
        showscale=False,
    ),
#    go.Scatter(
#        x = ['91 km 19 h','91 km 19 h'],
#        y = [0, len(z[1])],
#    )
]

layout = go.Layout(
    xaxis = dict( tickangle=90),
)

figure = dict(data=hm, layout=layout)

plot(figure, filename='pictures/agents_undisciplined.html', include_plotlyjs=True, show_link=False)

'file:///home/vagrant/what-the-data/pictures/agents_undisciplined.html'

In [4]:
IFrame('pictures/agents_undisciplined.html', width=700, height=400)

### Cooperating agents  (red : not enough fuel)

In [45]:
import json
import os

with open(os.path.join('terawatt_simulation', 'agents_cooperation.dat')) as json_data:
    data = json.load(json_data)


z = []
x = []
for i,data_agent in enumerate(data):
    c1 = []
    c2 = []
    x.append(str(data_agent[1]['car1_distance_work'])+' km '+str(data_agent[1]['charge_car1_start'])+' h') #+str(i*2))
    x.append(str(data_agent[1]['car2_distance_work'])+' km '+str(data_agent[1]['max_energy_timestamp']/3600)+' h') #+str(i*2+1)) 
    for d in data_agent:
        c1.append(int(d['car1_drive_success']))
        c2.append(int(d['car2_drive_success']))
    z.append(c1)
    z.append(c2)

colorscale = [[0, 'red'], [1, '#001f3f']]
hm = [
    go.Heatmap(
        z=numpy.transpose(z),
        x=numpy.transpose(x),
        colorscale=colorscale,
        showscale=False,
    )
]

layout = go.Layout(
    xaxis = dict( tickangle=90),
)

figure = dict(data=hm, layout=layout)

plot(figure, filename='pictures/agents_cooperation.html', include_plotlyjs=True, show_link=False)

'file:///home/vagrant/what-the-data/pictures/agents_cooperation.html'

In [46]:
IFrame('pictures/agents_cooperation.html', width=700, height=400)

### Undisciplined agents - Deficit by provider

In [5]:
import json
import os

with open(os.path.join('terawatt_simulation_real_data', 'agents_undisciplined_provider.dat')) as json_data:
    data = json.load(json_data)


z = []
x = []
for i,data_agent in enumerate(data):
    c1 = []
    c2 = []
    x.append(str(data_agent[1]['car1_distance_work'])+' km '+str(data_agent[1]['charge_car1_start'])+' h') #+str(i*2))
    x.append(str(data_agent[1]['car2_distance_work'])+' km '+str(data_agent[1]['charge_car2_start'])+' h') #+str(i*2+1)) 
    for d in data_agent:
        c1.append(float(d['car1_energy']))
        c2.append(float(d['car2_energy']))
    z.append(c1)
    z.append(c2)

colorscale = [[1, '#001f3f'], [0, 'red']]
hm = [
    go.Heatmap(
        z=numpy.transpose(z),
        x=numpy.transpose(x),
        #colorscale=colorscale,
        #showscale=False,
    )
]

layout = go.Layout(
    xaxis = dict( tickangle=90),
)

figure = dict(data=hm, layout=layout)

plot(figure, filename='pictures/agents_undisciplined_provider.html', include_plotlyjs=True, show_link=False)

'file:///home/vagrant/what-the-data/pictures/agents_undisciplined_provider.html'

In [6]:
IFrame('pictures/agents_undisciplined_provider.html', width=700, height=400)

### Cooperating agents - Deficit by provider

In [11]:
import json
import os

with open(os.path.join('terawatt_simulation', 'agents_cooperation_provider.dat')) as json_data:
    data = json.load(json_data)


z = []
x = []
for i,data_agent in enumerate(data):
    c1 = []
    c2 = []
    x.append(str(data_agent[1]['car1_distance_work'])+' km '+str(data_agent[1]['charge_car1_start'])+' h') #+str(i*2))
    x.append(str(data_agent[1]['car2_distance_work'])+' km '+str(data_agent[1]['max_energy_timestamp']/3600)+' h') #+str(i*2+1))
    for d in data_agent:
        c1.append(float(d['car1_energy']))
        c2.append(float(d['car2_energy']))
    z.append(c1)
    z.append(c2)

colorscale = [[1, '#001f3f'], [0, 'red']]
hm = [
    go.Heatmap(
        z=numpy.transpose(z),
        x=numpy.transpose(x),
        #colorscale=colorscale,
        #showscale=False,
    )
]

layout = go.Layout(
    xaxis = dict( tickangle=90),
)

figure = dict(data=hm, layout=layout)

plot(figure, filename='pictures/agents_cooperation_provider.html', include_plotlyjs=True, show_link=False)

'file:///home/vagrant/what-the-data/pictures/agents_cooperation_provider.html'

In [12]:
IFrame('pictures/agents_cooperation_provider.html', width=700, height=400)